In [81]:
# import openpyxl
# from DataModule import BasicMessage
# from openpyxl.utils import get_column_letter
#
#
# class FirstDeal:
#     def __init__(self, filepath):
#         self.file_path = filepath
#         self.company_sheet_detail = {}  # 所有数据
#         self.time_data = set()  # 可选日期
#
#         # 尝试打开文件，得到wb
#         try:
#             self.wb = openpyxl.load_workbook(filepath,data_only=True)
#             self.workbook_sheets_names = self.wb.sheetnames
#         except:
#             print("somethings wrong")
#         self.deal_sheets()
#         self.get_time_data()
#
#     def deal_sheets(self):
#         """
#         处理文件相关sheets name 数据规整
#         company_sheet_detail  :  各公司简写名字的对应有的表名
#         :return: self.company_sheet_detail
#         """
#
#         # 公司简写
#         company_abb = list(BasicMessage.company_abbreviation.values())
#         # company_all = list(BasicMessage.company_abbreviation.keys())  #全写
#
#         # 构建公司字典
#         for x in company_abb:
#             if x not in self.company_sheet_detail:
#                 self.company_sheet_detail[x] = {}
#             else:
#                 pass
#         # 读取所有表格中所有的信息，并按照预定格式组合
#         for item_company_abb in company_abb:  #
#             for item_sheet_name in self.workbook_sheets_names:
#                 if item_company_abb in item_sheet_name:  # 找到对应表
#                     # 读取单张表格 获取未经处理的所有数据
#                     sheet_data = self.read_single_sheet_table(sheet_name=item_sheet_name)
#                     # 处理数据，得到组合数据
#                     finally_sheet_data = self.rebuilt_page_data(sheet_data)
#                     # 组合数据
#                     self.company_sheet_detail[item_company_abb][item_sheet_name] = finally_sheet_data
#         # 可接受返回的数据，也可以直接调取属性
#         return self.company_sheet_detail
#
#     def read_single_sheet_table(self, sheet_name):
#         """
#         获取单张表格的所有数据
#         :param sheet_name:
#         :return: data
#         """
#         ws = self.wb[sheet_name]
#         max_row_num = ws.max_row
#         max_col_num = ws.max_column
#         data = []
#         for row in range(2, max_row_num + 1):
#             row_item = []
#             for col in range(1, max_col_num):
#                 col_letter = get_column_letter(col)
#                 row_item.append(ws[col_letter + str(row)].value)
#             data.append(row_item)
#         return data
#
#     def rebuilt_page_data(self, data_list):
#         """
#         组装单张表格里面的所有数据
#         page_data = {
#             "Level1": {"code": {
#                                 "部门":"code"
#                                 },
#                         "data": [
#                                     ["科目代码","部门","项目","本期借方发生","本期借方累积"],
#                                 ]
#                     },
#             "Level2": {"code": [], "data": []},
#             "Level3": {"code": [], "data": []},
#             "Level4": {"code": [], "data": []},
#             "Level5": {"code": [], "data": []},
#              }
#         :return: page_data
#         """
#         # 预设数据
#         page_data = {
#                     "Level1": {"code": {}, "data": []},
#                     "Level2": {"code": {}, "data": []},
#                     "Level3": {"code": {}, "data": []},
#                     "Level4": {"code": {}, "data": []},
#                     "Level5": {"code": {}, "data": []},
#                      }
#         # 有效一级标题
#         useful_Level_1 = ["6601", "6602", "6603"]
#
#         for data_index in data_list:
#             # 遍历所有数据
#             if data_index is None or data_index[0] is None:
#                 # 无效数据第一批次就跳过
#                 pass
#             else:
#                 for useful_Level in useful_Level_1:
#                     # 只针对有效一级标题下的所有数据
#                     if useful_Level in data_index[0]:
#                         # 有效数据
#                         # ["科目代码","部门","项目","本期借方发生","本期借方累积"]
#                         data_detail = [data_index[0], data_index[1], data_index[2], data_index[8], data_index[10]]
#                         # 组织一级标题及数据
#                         if len(data_index[0]) == 4 and data_index[2] is None:
#                             page_data["Level1"]["code"][data_index[1]] = data_index[0]
#                         elif len(data_index[0]) == 4 and data_index[2] is not None:
#                             page_data["Level1"]["data"].append(data_detail)
#
#                         # 组织二级
#                         elif len(data_index[0]) == 7 and data_index[2] is None:
#                             page_data["Level2"]["code"][data_index[1]] = data_index[0]
#                         elif len(data_index[0]) == 7 and data_index[2] is not None:
#                             page_data["Level2"]["data"].append(data_detail)
#
#                         # 组织三级
#                         elif len(data_index[0]) == 10 and data_index[2] is None:
#                             page_data["Level3"]["code"][data_index[1]] = data_index[0]
#                         elif len(data_index[0]) == 10 and data_index[2] is not None:
#                             page_data["Level3"]["data"].append(data_detail)
#
#                         # 组织四级
#                         elif len(data_index[0]) == 13 and data_index[2] is None:
#                             page_data["Level4"]["code"][data_index[1]] = data_index[0]
#                         elif len(data_index[0]) == 13 and data_index[2] is not None:
#                             page_data["Level4"]["data"].append(data_detail)
#
#                         # 组织五级
#                         elif len(data_index[0]) == 16 and data_index[2] is None:
#                             page_data["Level5"]["code"][data_index[1]] = data_index[0]
#                         elif len(data_index[0]) == 16 and data_index[2] is not None:
#                             page_data["Level5"]["data"].append(data_detail)
#                     else:
#                         continue
#         return page_data
#
#     def get_time_data(self):
#         """
#         获取表单中存在的有效数据，方便后续筛选的时候使用
#         :return:self.time_data
#         """
#         for company in self.company_sheet_detail.keys():
#             for company_date in self.company_sheet_detail[company].keys():
#                 self.time_data.add(company_date[-4:])
#         self.time_data = list(self.time_data)
#         self.time_data.sort()
#         return self.time_data
#
#     def calculate_year_basis(self):
#         """
#         计算环比，同比，
#         :return:
#         """
#         pass

In [82]:
# a = FirstDeal("/Users/MikeImac/Desktop/FinanceTool/经营检测表（费用明细表）数据底稿.xlsx" )

In [83]:
# a.company_sheet_detail["之了成教"]["之了成教2204"]["Level3"]["data"]

[['6602.01.01', '论文项目_论文项目组', '排版', 3834.67, 4863.33],
 ['6602.06.06', '之了会计研究院_Python项目组', '编程', 1884, 777.33],
 ['6602.06.06', '论文项目_论文项目组', '排版', 5636, 4909.33],
 ['6602.08.01', '行政中心', '综合', 2768, 961.33],
 ['6602.08.02', '行政中心', '综合', 2879.33, 3406.67],
 ['6602.08.03', '行政中心', '综合', 6608.67, 2407.33],
 ['6602.09.01', '之了会计研究院_Python项目组', '编程', 3794, 2894],
 ['6602.12.02', '之了会计研究院_Python项目组', '编程', 3615.33, 2328.67],
 ['6602.12.02', '论文项目_论文项目组', '排版', 4311.33, 5071.33],
 ['6602.16.04', '财务中心_资金部', '综合', 0, 4174.67],
 ['6602.17.06', '财务中心_资金部', '综合', 0, 2549.33],
 ['6602.23.06', '总裁办', '查重', 0, 1567.33],
 ['6602.23.07', '市场中心_电商渠道', '排版', 0, 1400.67],
 ['6602.23.07', '研发中心_研发中心', '排版', 1625.33, 3023.33],
 ['6602.23.07', '论文项目_论文项目组', '排版', 2044.67, 5461.33],
 ['6602.23.07', '论文项目_论文项目组', '查重', 4316, 2634.67],
 ['6602.97.01', '之了会计研究院_Python项目组', '编程', -25.72, 0],
 ['6602.97.01', '论文项目_论文项目', '排版', 5299.33, 3130.67],
 ['6602.97.01', '论文项目_论文项目', '排版', 4406, 2188.67],
 ['6602.97.04'

In [84]:
# def get_compare_date(data_string:str):
#         date_data = data_string[-4:]
#         company_head = data_string[:-4]
#         tong_bi = company_head + str(int(date_data)-100)
#         if date_data[-2:] == "01":
#             circle_bi = company_head + str(int(date_data[0:2])-1)+str(12)
#         else:
#             circle_bi = company_head + str(int(date_data)-1)
#         return circle_bi,tong_bi
#
# def get_target_data(data_dict,company,date,code,depart):
#     level = "Level1"
#     if len(code) == 4:
#         level = "Level1"
#     elif len(code) == 7:
#         level = "Level2"
#     elif len(code) == 10:
#         level = "Level3"
#     elif len(code) == 13:
#         level = "Level4"
#     elif len(code) == 16:
#         level = "Level5"
#
#     try :
#         data_temp = data_dict[company][date][level]["data"]
#     except:
#         return None
#
#     if data_temp:
#         exist_flag = 0
#         for item in data_temp:
#             if item[0] == code and depart == item[1]:
#                 exist_flag = 1
#                 return item
#         if exist_flag == 0:
#             return None
#
# level_list  = ["Level1", "Level2", "Level3", "Level4", "Level5"]
#
# for company_index in a.company_sheet_detail:
#     for date_index in a.company_sheet_detail[company_index]:
#         for level_meg in level_list:
#             # print(a.company_sheet_detail[company_index][date_index][level_meg]["data"])
#             if a.company_sheet_detail[company_index][date_index][level_meg]["data"]:
#                 # print(f'a.company_sheet_detail[{company_index}][{date_index}][{level_meg}]["data"]')
#                 for singe_item in a.company_sheet_detail[company_index][date_index][level_meg]["data"]:
#                     data_index = a.company_sheet_detail[company_index][date_index][level_meg]["data"].index(singe_item)
#                     # 当月同
#                     num_1 = singe_item[3]
#                     num_2 = get_target_data(data_dict = a.company_sheet_detail,
#                                             company=company_index,
#                                             date=get_compare_date(date_index)[1],
#                                             code=singe_item[0],
#                                             depart=singe_item[1],
#                                             )
#                     #当月环
#                     num_3 = get_target_data(data_dict = a.company_sheet_detail,
#                                             company=company_index,
#                                             date=get_compare_date(date_index)[0],
#                                             code=singe_item[0],
#                                             depart=singe_item[1]
#                                             ,)
#
#                     if num_2 is None or int(num_2[3]) == 0:
#                         tong_data = "0"
#                     else:
#                         tong_data = str((int(num_1) - int(num_2[3]))/int(num_2[3]))
#
#                     if num_3 is None or int(num_3[3]) == 0:
#                         circle_data = "0"
#                     else:
#                         circle_data = str((int(num_1) - int(num_3[3]))/int(num_3[3]))
#
#                     # 当年同
#                     num_a = singe_item[4]
#                     num_b = get_target_data(data_dict = a.company_sheet_detail,
#                                             company=company_index,
#                                             date=get_compare_date(date_index)[1],
#                                             code=singe_item[0],
#                                             depart=singe_item[1],
#                                             )
#                     if num_b is None or int(num_b[4]) == 0:
#                         year_tong_data = "0"
#                     else:
#                         year_tong_data = str((int(num_a) - int(num_b[4]) / int(num_b[4])))
#
#                     new_singe_item = singe_item[:]
#                     new_singe_item.append(tong_data)
#                     new_singe_item.append(circle_data)
#                     new_singe_item.append(year_tong_data)
#                     a.company_sheet_detail[company_index][date_index][level_meg]["data"][data_index] = new_singe_item
#
#


In [87]:
#!/usr/bin/env python
# -*- coding: UTF-8 -*-
"""
@Project ：FinanceTool
@File    ：ReadExcel.py
@Author  ：朱桃禾 MikePy
@Date    ：2022/10/8 10:47 AM
"""

import openpyxl
from DataModule import BasicMessage
from openpyxl.utils import get_column_letter


class FirstDeal:
    def __init__(self, filepath):
        self.file_path = filepath
        self.company_sheet_detail = {}  # 所有数据
        self.time_data = set()  # 可选日期

        # 尝试打开文件，得到wb
        try:
            self.wb = openpyxl.load_workbook(filepath, data_only=True)
            self.workbook_sheets_names = self.wb.sheetnames
        except:
            print("somethings wrong")
        self.deal_sheets()
        self.get_time_data()
        self.calculate_year_basis()

    def deal_sheets(self):
        """
        处理文件相关sheets name 数据规整
        company_sheet_detail  :  各公司简写名字的对应有的表名
        :return: self.company_sheet_detail
        """

        # 公司简写
        company_abb = list(BasicMessage.company_abbreviation.values())
        # company_all = list(BasicMessage.company_abbreviation.keys())  #全写

        # 构建公司字典
        for x in company_abb:
            if x not in self.company_sheet_detail:
                self.company_sheet_detail[x] = {}
            else:
                pass
        # 读取所有表格中所有的信息，并按照预定格式组合
        for item_company_abb in company_abb:  #
            for item_sheet_name in self.workbook_sheets_names:
                if item_company_abb in item_sheet_name:  # 找到对应表
                    # 读取单张表格 获取未经处理的所有数据
                    sheet_data = self.read_single_sheet_table(sheet_name=item_sheet_name)
                    # 处理数据，得到组合数据
                    finally_sheet_data = self.rebuilt_page_data(sheet_data)
                    # 组合数据
                    self.company_sheet_detail[item_company_abb][item_sheet_name] = finally_sheet_data
        # 可接受返回的数据，也可以直接调取属性
        return self.company_sheet_detail

    def read_single_sheet_table(self, sheet_name):
        """
        获取单张表格的所有数据
        :param sheet_name:
        :return: data
        """
        ws = self.wb[sheet_name]
        max_row_num = ws.max_row
        max_col_num = ws.max_column
        data = []
        for row in range(2, max_row_num + 1):
            row_item = []
            for col in range(1, max_col_num):
                col_letter = get_column_letter(col)
                row_item.append(ws[col_letter + str(row)].value)
            data.append(row_item)
        return data

    def rebuilt_page_data(self, data_list):
        """
        组装单张表格里面的所有数据
        page_data = {
            "Level1": {"code": {
                                "部门":"code"
                                },
                        "data": [
                                    ["科目代码","部门","项目","本期借方发生","本期借方累积"],
                                ]
                    },
            "Level2": {"code": [], "data": []},
            "Level3": {"code": [], "data": []},
            "Level4": {"code": [], "data": []},
            "Level5": {"code": [], "data": []},
             }
        :return: page_data
        """
        # 预设数据
        page_data = {
            "Level1": {"code": {}, "data": []},
            "Level2": {"code": {}, "data": []},
            "Level3": {"code": {}, "data": []},
            "Level4": {"code": {}, "data": []},
            "Level5": {"code": {}, "data": []},
        }
        # 有效一级标题
        useful_Level_1 = ["6601", "6602", "6603"]

        for data_index in data_list:
            # 遍历所有数据
            if data_index is None or data_index[0] is None:
                # 无效数据第一批次就跳过
                pass
            else:
                for useful_Level in useful_Level_1:
                    # 只针对有效一级标题下的所有数据
                    if useful_Level in data_index[0]:
                        # 有效数据
                        # ["科目代码","部门","项目","本期借方发生","本期借方累积"]
                        data_detail = [data_index[0], data_index[1], data_index[2], data_index[7], data_index[9]]
                        # 组织一级标题及数据
                        if len(data_index[0]) == 4 and data_index[2] is None:
                            page_data["Level1"]["code"][data_index[1]] = data_index[0]
                        elif len(data_index[0]) == 4 and data_index[2] is not None:
                            page_data["Level1"]["data"].append(data_detail)

                        # 组织二级
                        elif len(data_index[0]) == 7 and data_index[2] is None:
                            page_data["Level2"]["code"][data_index[1]] = data_index[0]
                        elif len(data_index[0]) == 7 and data_index[2] is not None:
                            page_data["Level2"]["data"].append(data_detail)

                        # 组织三级
                        elif len(data_index[0]) == 10 and data_index[2] is None:
                            page_data["Level3"]["code"][data_index[1]] = data_index[0]
                        elif len(data_index[0]) == 10 and data_index[2] is not None:
                            page_data["Level3"]["data"].append(data_detail)

                        # 组织四级
                        elif len(data_index[0]) == 13 and data_index[2] is None:
                            page_data["Level4"]["code"][data_index[1]] = data_index[0]
                        elif len(data_index[0]) == 13 and data_index[2] is not None:
                            page_data["Level4"]["data"].append(data_detail)

                        # 组织五级
                        elif len(data_index[0]) == 16 and data_index[2] is None:
                            page_data["Level5"]["code"][data_index[1]] = data_index[0]
                        elif len(data_index[0]) == 16 and data_index[2] is not None:
                            page_data["Level5"]["data"].append(data_detail)
                    else:
                        continue
        return page_data

    def get_time_data(self):
        """
        获取表单中存在的有效数据，方便后续筛选的时候使用
        :return:self.time_data
        """
        for company in self.company_sheet_detail.keys():
            for company_date in self.company_sheet_detail[company].keys():
                self.time_data.add(company_date[-4:])
        self.time_data = list(self.time_data)
        self.time_data.sort()
        return self.time_data

    def calculate_year_basis(self):
        """
        计算环比，同比，
        :return:
        """
        level_list = ["Level1", "Level2", "Level3", "Level4", "Level5"]

        for company_index in self.company_sheet_detail:
            for date_index in self.company_sheet_detail[company_index]:
                for level_meg in level_list:
                    if self.company_sheet_detail[company_index][date_index][level_meg]["data"]:
                        for singe_item in self.company_sheet_detail[company_index][date_index][level_meg]["data"]:
                            data_index = self.company_sheet_detail[company_index][date_index][level_meg]["data"].index(
                                singe_item)
                            # 当月同
                            num_1 = singe_item[3]
                            num_2 = self.get_target_data(data_dict=self.company_sheet_detail,
                                                         company=company_index,
                                                         date=self.get_compare_date(date_index)[1],
                                                         code=singe_item[0],
                                                         depart=singe_item[1],
                                                         )
                            # 当月环
                            num_3 = self.get_target_data(data_dict=self.company_sheet_detail,
                                                         company=company_index,
                                                         date=self.get_compare_date(date_index)[0],
                                                         code=singe_item[0],
                                                         depart=singe_item[1]
                                                         , )

                            if num_2 is None or int(num_2[3]) == 0:
                                tong_data = "0"
                            else:
                                tong_data = str((int(num_1) - int(num_2[3])) / int(num_2[3]))

                            if num_3 is None or int(num_3[3]) == 0:
                                circle_data = "0"
                            else:
                                circle_data = str((int(num_1) - int(num_3[3])) / int(num_3[3]))

                            # 当年同
                            num_a = singe_item[4]
                            num_b = self.get_target_data(data_dict=self.company_sheet_detail,
                                                         company=company_index,
                                                         date=self.get_compare_date(date_index)[1],
                                                         code=singe_item[0],
                                                         depart=singe_item[1],
                                                         )
                            if num_b is None or int(num_b[4]) == 0:
                                year_tong_data = "0"
                            else:
                                year_tong_data = str((int(num_a) - int(num_b[4]) / int(num_b[4])))

                            new_singe_item = singe_item[:]
                            new_singe_item.append(tong_data)
                            new_singe_item.append(circle_data)
                            new_singe_item.append(year_tong_data)
                            self.company_sheet_detail[company_index][date_index][level_meg]["data"][
                                data_index] = new_singe_item

    def get_compare_date(self, data_string: str):
        date_data = data_string[-4:]
        company_head = data_string[:-4]
        tong_bi = company_head + str(int(date_data) - 100)
        if date_data[-2:] == "01":
            circle_bi = company_head + str(int(date_data[0:2]) - 1) + str(12)
        else:
            circle_bi = company_head + str(int(date_data) - 1)
        return circle_bi, tong_bi

    def get_target_data(self, data_dict, company, date, code, depart):
        level = "Level1"
        if len(code) == 4:
            level = "Level1"
        elif len(code) == 7:
            level = "Level2"
        elif len(code) == 10:
            level = "Level3"
        elif len(code) == 13:
            level = "Level4"
        elif len(code) == 16:
            level = "Level5"

        try:
            data_temp = data_dict[company][date][level]["data"]
        except:
            return None

        if data_temp:
            exist_flag = 0
            for item in data_temp:
                if item[0] == code and depart == item[1]:
                    exist_flag = 1
                    return item
            if exist_flag == 0:
                return None

In [86]:
# a.company_sheet_detail["之了成教"]["之了成教2206"]

{'Level1': {'code': {'管理费用': '6602', '财务费用': '6603'}, 'data': []},
 'Level2': {'code': {'薪酬': '6602.01',
   '社会保险费': '6602.02',
   '住房公积金': '6602.03',
   '职工福利费': '6602.06',
   '职场费用': '6602.08',
   '办公费': '6602.09',
   '加班费': '6602.12',
   '清洁费': '6602.14',
   '邮电通讯费': '6602.16',
   '交通费': '6602.17',
   '劳动保护费': '6602.21',
   '咨询及服务费': '6602.23',
   '维修费': '6602.24',
   '宣传费': '6602.26',
   '长期待摊费用摊销': '6602.30',
   '研发支出': '6602.35',
   '平台费': '6602.97',
   '银行手续费': '6603.01',
   '利息收入': '6603.03'},
  'data': [['6602.02',
    '电商中心_电商客服组',
    '综合',
    5726.67,
    6300.67,
    '0',
    '0.7949843260188088',
    '0'],
   ['6602.02',
    '之了会计研究院_Python项目组',
    '编程',
    2068,
    5800.67,
    '0',
    '-0.36133415688696724',
    '0'],
   ['6602.02',
    '继续教育学历提升项目',
    '综合',
    5133.33,
    5700,
    '0',
    '2.252851711026616',
    '0'],
   ['6602.02',
    '论文项目_论文项目组',
    '排版',
    5847.33,
    4485.33,
    '0',
    '0.032673966796185096',
    '0'],
   ['6602.03',
    '电商中心_

In [88]:
a = FirstDeal("/Users/MikeImac/Desktop/FinanceTool/经营检测表（费用明细表）数据底稿.xlsx" )

In [89]:
a.company_sheet_detail

{'环宇': {'环宇2104': {'Level1': {'code': {'销售费用': '6601',
     '管理费用': '6602',
     '财务费用': '6603'},
    'data': []},
   'Level2': {'code': {'薪酬': '6602.01',
     '社会保险费': '6602.02',
     '住房公积金': '6602.03',
     '商业保险费': '6602.04',
     '职工福利费': '6602.06',
     '职场费用': '6602.08',
     '办公费': '6602.09',
     '加班费': '6602.12',
     '团队建设费': '6602.13',
     '清洁费': '6602.14',
     '招聘费': '6602.15',
     '邮电通讯费': '6602.16',
     '交通费': '6602.17',
     '会议费': '6602.19',
     '培训费': '6602.20',
     '劳动保护费': '6602.21',
     '业务招待费': '6602.22',
     '咨询及服务费': '6602.23',
     '维修费': '6602.24',
     '宣传费': '6602.26',
     '固定资产折旧费': '6602.28',
     '长期待摊费用摊销': '6602.30',
     '劳务费': '6602.32',
     '安全保卫费': '6602.36',
     '平台费': '6602.97',
     '差旅费': '6602.18',
     '无形资产摊销费': '6602.29',
     '包装费': '6602.33',
     '研发支出': '6602.35',
     '银行手续费': '6603.01',
     '利息收入': '6603.03'},
    'data': [['6601.02', '销售中心_销售中心', '综合', 0, 0, '0', '0', '0'],
     ['6601.02', '销售中心_副总裁助理', '综合', 0, 0, '0', '

In [90]:
a.company_sheet_detail["之了成教"]["之了成教2206"]

{'Level1': {'code': {'管理费用': '6602', '财务费用': '6603'}, 'data': []},
 'Level2': {'code': {'薪酬': '6602.01',
   '社会保险费': '6602.02',
   '住房公积金': '6602.03',
   '职工福利费': '6602.06',
   '职场费用': '6602.08',
   '办公费': '6602.09',
   '加班费': '6602.12',
   '清洁费': '6602.14',
   '邮电通讯费': '6602.16',
   '交通费': '6602.17',
   '劳动保护费': '6602.21',
   '咨询及服务费': '6602.23',
   '维修费': '6602.24',
   '宣传费': '6602.26',
   '长期待摊费用摊销': '6602.30',
   '研发支出': '6602.35',
   '平台费': '6602.97',
   '银行手续费': '6603.01',
   '利息收入': '6603.03'},
  'data': [['6602.02', '电商中心_电商客服组', '综合', 0, 5728.67, '0', '0', '0'],
   ['6602.02', '之了会计研究院_Python项目组', '编程', 0, 4574, '0', '0', '0'],
   ['6602.02', '继续教育学历提升项目', '综合', 0, 4778, '0', '0', '0'],
   ['6602.02', '论文项目_论文项目组', '排版', 0, 1198.67, '0', '0', '0'],
   ['6602.03', '电商中心_电商客服组', '综合', 0, 1084, '0', '0', '0'],
   ['6602.03', '之了会计研究院_Python项目组', '编程', 0, 1158, '0', '0', '0'],
   ['6602.03', '论文项目_论文项目组', '排版', 0, 809.33, '0', '0', '0'],
   ['6602.14', '行政中心', '综合', 0, 6140.67, '0